In [1]:
import os
import sys
import json
import copy
import re
from tqdm import tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
w_dir

'/work/jerry_se'

In [2]:
sys.path.append(w_dir+'/src')

In [3]:
from src.std import *
from src import config

In [4]:
config.FGC_DEV

PosixPath('/work/jerry_se/data/FGC/FGC_release_1.7.13/FGC_release_all_dev.json')

In [5]:
import torch
from tqdm import tqdm_notebook as tqdm
from transformers import BertTokenizer, PreTrainedTokenizer, BertModel, BertForMaskedLM, BertForQuestionAnswering
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch.nn.utils.rnn import pad_sequence 
import torchvision
import torch.nn as nn
from transformers.optimization import AdamW
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
device = torch.device("cuda")
import pandas as pd

I0827 06:03:31.765957 140332525516608 file_utils.py:39] PyTorch version 1.2.0+cu92 available.


In [6]:
from src.preprocess import AttnDataset, data_preprocessing

I0827 06:03:33.811166 140332525516608 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [7]:
validation_data = json_load("data/FGC/FGC_release_1.7.13/FGC_release_all_dev.json")
training_data = json_load("data/FGC/FGC_release_1.7.13/FGC_release_all_train.json")
test_data = json_load("data/FGC/FGC_release_1.7.13/FGC_release_all_test.json")

In [8]:
attentionDataset = AttnDataset(training_data, 'fgc', False, 1)

100%|██████████| 882/882 [00:00<00:00, 3454.25it/s]


871


In [21]:
train_set = attentionDataset[:5]

In [12]:
train_set[0]['input_ids']

tensor([[ 101, 5722,  691, 1786, 4638, 5439, 2157, 1762, 1525,  136,  102, 1649,
          859,  753, 2399, 8020, 8423, 8161, 2399, 8021, 8024,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [19]:
#data = json_load(config.FGC_DEV)
#examples = data_preprocessing(data, 3)

In [22]:
dataloader_train = DataLoader(train_set, shuffle=True, batch_size=2)

In [23]:
a = iter(dataloader_train).next()

In [30]:
a['label']

tensor([[0],
        [0]])

In [34]:
for batch in dataloader_train:
    print(batch['input_ids'].device)
    for t_n, t in batch.items():
        batch[t_n] = t.to(device)
    print(batch['input_ids'].device)

cpu
cuda:0


In [38]:
input_ids = a['input_ids'].view(-1, 300)
attention_mask = a['attention_mask'].view(-1, 300)
token_type_ids = a['token_type_ids'].view(-1, 300)

In [62]:
ip_ids = input_ids[1].view(-1, 300)
a_m = attention_mask[1].view(-1, 300)
tt_ids = token_type_ids[1].view(-1, 300)

In [81]:
a['input_ids'].view(-1, 300)[1].shape

torch.Size([300])

In [72]:
hidden_state_1, pooler_output_1 = bert(ip_ids, a_m, tt_ids)

In [46]:
pooler_output

tensor([[ 0.9983,  0.9997,  0.9999,  ..., -0.9993, -0.9380,  0.8870],
        [-0.3392,  1.0000, -0.9980,  ..., -0.1359, -0.4057,  0.9860],
        [ 1.0000, -0.2586,  1.0000,  ..., -0.9997, -0.9993, -0.6872]],
       grad_fn=<TanhBackward>)

In [70]:
pooler_output[1]

tensor([-3.3920e-01,  1.0000e+00, -9.9798e-01,  9.9959e-01,  9.9687e-01,
         9.9880e-01, -9.9680e-01, -9.9540e-01,  9.6463e-01, -4.9464e-01,
         9.9944e-01, -1.0000e+00, -9.9717e-01, -9.9960e-01,  9.8542e-01,
        -9.8713e-01, -9.1356e-01,  9.8376e-01,  9.0224e-01,  6.4627e-01,
         1.0000e+00, -9.9801e-01, -9.9955e-01,  9.9943e-01,  7.2418e-01,
         9.2784e-01,  9.6677e-01, -9.5308e-01, -1.0000e+00, -9.6481e-01,
         9.5793e-01,  6.8416e-01,  6.5253e-01, -1.0000e+00, -9.9443e-01,
         9.9790e-01, -9.4310e-01,  3.7721e-01, -5.2735e-01, -7.8606e-01,
         4.2142e-02, -9.9719e-01,  9.9827e-01,  8.0791e-01, -9.9329e-01,
        -9.7075e-01, -9.9979e-01, -9.9939e-01,  6.8449e-01,  1.0000e+00,
         2.2763e-03, -9.7740e-01, -9.9990e-01,  9.9989e-01, -9.9100e-01,
        -4.3746e-01, -9.9524e-01,  6.9199e-01,  9.9988e-01,  9.8963e-01,
         8.1561e-01, -9.9983e-01,  9.8238e-01, -9.4690e-01,  9.9850e-01,
        -9.6268e-01,  2.9591e-01,  9.9918e-01,  1.0

In [73]:
pooler_output_1

tensor([[-3.3920e-01,  1.0000e+00, -9.9798e-01,  9.9959e-01,  9.9687e-01,
          9.9880e-01, -9.9680e-01, -9.9540e-01,  9.6463e-01, -4.9464e-01,
          9.9944e-01, -1.0000e+00, -9.9717e-01, -9.9960e-01,  9.8542e-01,
         -9.8713e-01, -9.1356e-01,  9.8376e-01,  9.0224e-01,  6.4627e-01,
          1.0000e+00, -9.9801e-01, -9.9955e-01,  9.9943e-01,  7.2418e-01,
          9.2784e-01,  9.6677e-01, -9.5308e-01, -1.0000e+00, -9.6481e-01,
          9.5793e-01,  6.8416e-01,  6.5253e-01, -1.0000e+00, -9.9443e-01,
          9.9790e-01, -9.4310e-01,  3.7721e-01, -5.2735e-01, -7.8606e-01,
          4.2142e-02, -9.9719e-01,  9.9827e-01,  8.0791e-01, -9.9329e-01,
         -9.7075e-01, -9.9979e-01, -9.9939e-01,  6.8449e-01,  1.0000e+00,
          2.2763e-03, -9.7740e-01, -9.9990e-01,  9.9989e-01, -9.9100e-01,
         -4.3746e-01, -9.9524e-01,  6.9199e-01,  9.9988e-01,  9.8963e-01,
          8.1561e-01, -9.9983e-01,  9.8238e-01, -9.4690e-01,  9.9850e-01,
         -9.6268e-01,  2.9591e-01,  9.